In [1]:
import librosa
import librosa.display
import os
from scipy.io import loadmat
import numpy as np
import matplotlib.pyplot as plt

/usr/local/lib/python3.6/dist-packages/numba/errors.py:137: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [2]:
#currdir= os.getcwd()
rootdir = '/home/taejoon/PhysioNetChallenge'
input_directory = os.path.join(rootdir, 'Training_WFDB')
output_directory = os.path.join(rootdir, 'Mel_data')
#save_directory = os.path.join(currdir, '')
if not os.path.isdir(input_directory):
        os.mkdir(input_directory)
if not os.path.isdir(output_directory):
        os.mkdir(output_directory)        

In [3]:
# Find files
input_files = []
for f in os.listdir(input_directory):
    if os.path.isfile(os.path.join(input_directory, f)) and not f.lower().startswith('.') and f.lower().endswith('mat'):
        input_files.append(f)

In [4]:
input_files

['A3066.mat',
 'A4872.mat',
 'A2971.mat',
 'A4463.mat',
 'A1499.mat',
 'A4135.mat',
 'A0099.mat',
 'A5945.mat',
 'A0101.mat',
 'A6165.mat',
 'A6623.mat',
 'A1510.mat',
 'A5206.mat',
 'A6861.mat',
 'A1695.mat',
 'A4785.mat',
 'A5838.mat',
 'A5764.mat',
 'A0736.mat',
 'A4505.mat',
 'A2486.mat',
 'A4292.mat',
 'A1249.mat',
 'A2787.mat',
 'A3112.mat',
 'A2502.mat',
 'A0561.mat',
 'A0354.mat',
 'A3610.mat',
 'A1095.mat',
 'A1237.mat',
 'A0290.mat',
 'A3675.mat',
 'A1245.mat',
 'A2951.mat',
 'A2442.mat',
 'A2273.mat',
 'A4847.mat',
 'A5914.mat',
 'A2457.mat',
 'A4652.mat',
 'A3839.mat',
 'A5344.mat',
 'A3325.mat',
 'A5149.mat',
 'A1147.mat',
 'A4396.mat',
 'A4677.mat',
 'A3641.mat',
 'A3509.mat',
 'A5180.mat',
 'A1654.mat',
 'A6139.mat',
 'A6663.mat',
 'A5743.mat',
 'A4251.mat',
 'A3508.mat',
 'A2306.mat',
 'A2458.mat',
 'A3184.mat',
 'A3193.mat',
 'A2542.mat',
 'A5859.mat',
 'A0768.mat',
 'A5758.mat',
 'A0008.mat',
 'A5539.mat',
 'A1090.mat',
 'A4053.mat',
 'A2758.mat',
 'A3473.mat',
 'A275

In [5]:
np.shape(input_files)

(6877,)

### Mel Config

In [6]:
# n_mels = 128
# frame_length = 0.025 # 1초를 40개로?
# hop_length = 512
#hop_length = 512 -> 얘가 x축 bin 결정
#input_nfft = int(round(sr*frame_length))
# n_fft = 1024
sr = 500

fftsize = 1024
window = 1024
hop = 294
melBin = 224

def Mel(data, sr, fftsize, hop, window, melBin):
    # STFT
    S = librosa.core.stft(data, n_fft=fftsize, hop_length=hop, win_length = window)
    X = np.abs(S)
    # Mel basis
    mel_basis = librosa.filters.mel(sr, n_fft = fftsize, n_mels = melBin)
    # mal basis are multiplied to the STFT
    mel_S = np.dot(mel_basis, X)
    
    # log amplitude compression
    mel_S = np.log10(1+10*mel_S)
    mel_S = mel_S.astype(np.float32)
    
    
    return mel_S

In [7]:
def get_12ECG_features(header_data):
    tmp_hea = header_data[0].split(' ')
    ptID = tmp_hea[0]
    num_leads = int(tmp_hea[1])
    sample_Fs= int(tmp_hea[2])
    gain_lead = np.zeros(num_leads)
    for ii in range(num_leads):
        tmp_hea = header_data[ii+1].split(' ') # +1인 이유는 첫 행을 빼야하니
        gain_lead[ii] = int(tmp_hea[2].split('/')[0]) # 아마도 모든 행을 1000mV로 채울듯

    for iline in header_data:
        if iline.startswith('#Age'):
            tmp_age = iline.split(': ')[1].strip()
            age = int(tmp_age if tmp_age != 'NaN' else 57)
        elif iline.startswith('#Sex'):
            tmp_sex = iline.split(': ')[1]
            if tmp_sex.strip()=='Female':
                sex =1
            else:
                sex=0
        elif iline.startswith('#Dx'):
            label = iline.split(': ')[1].split(',')[0]
    return ptID, num_leads, sample_Fs, gain_lead, age, sex, label

In [8]:
for i in range(len(input_files)): # loops over subjects
    filename = os.path.join(input_directory, input_files[i])
    x = loadmat(filename)
    data = np.asarray(x['val'], dtype=np.float64)
    new_file = filename.replace('.mat', '.hea')
    input_header_file = os.path.join(new_file)
    save_name = output_directory + '/' + input_files[i].replace('.mat', '.npy') #filename.replace('.mat', '.npy')
    if not os.path.exists(os.path.dirname(save_name)):
        os.makedirs(os.path.dirname(save_name))

    with open(input_header_file,'r') as f:
        header_data=f.readlines()
    ptID, num_leads, sample_Fs, gain_lead, age, sex, label = get_12ECG_features(header_data)

    S = []
    for j in range(data.shape[0]): # loops over 12 leads
        tmp_lead = data[j]
        S1 = Mel(tmp_lead, sr, fftsize, hop, window, melBin)
        S1 = list(S1)
        S.append(S1)
    S = np.swapaxes(S, 0, 2)
    print(new_file, S.shape, save_name)
    np.save(save_name, S)

/home/taejoon/PhysioNetChallenge/Training_WFDB/A3066.hea (86, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3066.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4872.hea (19, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4872.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2971.hea (28, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2971.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4463.hea (19, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4463.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1499.hea (29, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1499.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4135.hea (99, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4135.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0099.hea (35, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0099.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5945.hea (89, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5945.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A3473.hea (30, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3473.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2753.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2753.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4523.hea (48, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4523.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2094.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2094.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5039.hea (57, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5039.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5423.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5423.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5538.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5538.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0912.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0912.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A0196.hea (23, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0196.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1122.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1122.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0629.hea (61, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0629.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2322.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2322.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6341.hea (86, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6341.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5319.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5319.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4773.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4773.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4583.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4583.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A6334.hea (25, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6334.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5084.hea (35, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5084.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4820.hea (60, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4820.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0416.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0416.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5625.hea (27, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5625.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3138.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3138.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1738.hea (32, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1738.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6782.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6782.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A2474.hea (94, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2474.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2248.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2248.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0269.hea (27, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0269.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4754.hea (23, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4754.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3887.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3887.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3857.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3857.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2426.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2426.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1300.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1300.npy
/home/taejoon/Ph

...

/home/taejoon/PhysioNetChallenge/Training_WFDB/A6566.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6566.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3281.hea (24, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3281.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1906.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1906.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6659.hea (59, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6659.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0622.hea (29, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0622.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6867.hea (19, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6867.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3219.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3219.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0513.hea (21, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0513.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A1274.hea (35, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1274.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0999.hea (23, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0999.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1391.hea (29, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1391.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4332.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4332.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3882.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3882.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2406.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2406.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3798.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3798.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4393.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4393.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A5497.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5497.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1101.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1101.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6672.hea (24, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6672.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4958.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4958.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2767.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2767.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2759.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2759.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0839.hea (26, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0839.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2477.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2477.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A4819.hea (22, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4819.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4805.hea (36, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4805.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4539.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4539.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4153.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4153.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2058.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2058.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2040.hea (23, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2040.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A2008.hea (22, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A2008.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A6189.hea (19, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A6189.npy
/home/taejoon/Ph

/home/taejoon/PhysioNetChallenge/Training_WFDB/A4643.hea (29, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4643.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A4067.hea (23, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A4067.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0516.hea (28, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0516.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A3706.hea (22, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A3706.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5257.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5257.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A0981.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A0981.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A5874.hea (18, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A5874.npy
/home/taejoon/PhysioNetChallenge/Training_WFDB/A1387.hea (29, 224, 12) /home/taejoon/PhysioNetChallenge/Mel_data/A1387.npy
/home/taejoon/Ph

In [9]:
np.max(S)

6.196763

In [10]:
S.max()

6.196763

In [11]:
S.min()

1.3273971

In [13]:
S.shape

(19, 224, 12)

In [12]:
S

array([[[5.2530246, 5.29269  , 4.4623866, ..., 5.668442 , 5.4774256,
         5.3673644],
        [4.869917 , 4.9835043, 4.345513 , ..., 5.264197 , 5.1055307,
         5.0036726],
        [4.7259374, 4.8680053, 4.3136387, ..., 4.8939567, 4.97057  ,
         4.962216 ],
        ...,
        [2.0678225, 2.074748 , 1.9071183, ..., 2.6676037, 2.3538096,
         2.2256973],
        [2.266033 , 2.1995845, 1.9226938, ..., 2.455216 , 2.511804 ,
         2.1160076],
        [2.1405933, 2.013649 , 1.9958125, ..., 2.2782044, 1.8568963,
         1.9633745]],

       [[5.554129 , 5.5470004, 4.149493 , ..., 5.9896903, 5.778081 ,
         5.6800985],
        [5.0615563, 5.029015 , 4.4739113, ..., 5.6539555, 5.3250513,
         5.1583014],
        [5.2713366, 5.3626294, 4.82568  , ..., 5.673319 , 5.5438614,
         5.489228 ],
        ...,
        [2.5074387, 2.339871 , 2.4056747, ..., 2.297943 , 2.2929785,
         2.4180214],
        [2.375787 , 2.4627762, 2.2945817, ..., 2.161034 , 2.344473 ,
   